# Adding a user-defined decision module to the turtles: The boids model

The [boids](https://en.wikipedia.org/wiki/Boids) (bird-oid) model has been invented by [Craig Reynolds](https://en.wikipedia.org/wiki/Craig_Reynolds_(computer_graphics)) in 1986 to simulate the flocking behavior of birds. It is based on 3 principles:
    
* separation: boids tend to avoid other boids that are too close,

* alignment: boids tend to align their velocity to boids that are not too close and not too far away,

* cohesion: bois tend to move towards boids that are too far away.

While these rules are essentially heuristic, they can be implemented defining three areas (repulsion, orientation, attraction) for each principle. 

* Boids change their orientation to get away from other boids in the repulsion area,

* Boids change their orientation and speed to match those of other boids in the orientation area,

* Boids change their orientation to get to other boids in the attraction area.

## Model parameters

The model parameters and their default values are defined

## Decision model

The decision model consists in changing the direction and speed of the boids according to the previously described rules.
To define a decision model, the modeler must define an object that extends `AbstractAgtDecisionModel` and implement the `decide` method.

## The simulation model

In the simulation model defined in our example, boids are initially located randomly in the environment with a random orientation and speed.

## Launch the HTML runner

Finally, we launch and configure the HTML runner 

In [1]:
#@repository("~/.m2/repository/")
#@dependency(group="fr.univ-artois.lgi2a", module="similar2logo-lib", version="1.0-SNAPSHOT")


import static java.lang.Math.*
import fr.univ_artois.lgi2a.similar.extendedkernel.libs.random.PRNG
import fr.univ_artois.lgi2a.similar.extendedkernel.libs.abstractimpl.AbstractAgtDecisionModel
import fr.univ_artois.lgi2a.similar.extendedkernel.simulationmodel.ISimulationParameters
import fr.univ_artois.lgi2a.similar.microkernel.AgentCategory
import fr.univ_artois.lgi2a.similar.microkernel.LevelIdentifier
import fr.univ_artois.lgi2a.similar.microkernel.SimulationTimeStamp
import fr.univ_artois.lgi2a.similar.microkernel.ISimulationModel.AgentInitializationData
import fr.univ_artois.lgi2a.similar.microkernel.agents.IGlobalState
import fr.univ_artois.lgi2a.similar.microkernel.agents.ILocalStateOfAgent
import fr.univ_artois.lgi2a.similar.microkernel.agents.IPerceivedData
import fr.univ_artois.lgi2a.similar.microkernel.influences.InfluencesMap
import fr.univ_artois.lgi2a.similar.microkernel.levels.ILevel
import fr.univ_artois.lgi2a.similar2logo.kernel.initializations.AbstractLogoSimulationModel
import fr.univ_artois.lgi2a.similar2logo.kernel.model.LogoSimulationParameters
import fr.univ_artois.lgi2a.similar2logo.kernel.model.Parameter
import fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleAgentCategory
import fr.univ_artois.lgi2a.similar2logo.kernel.model.agents.turtle.TurtleFactory
import fr.univ_artois.lgi2a.similar2logo.kernel.model.influences.ChangeDirection
import fr.univ_artois.lgi2a.similar2logo.kernel.model.influences.ChangeSpeed
import fr.univ_artois.lgi2a.similar2logo.kernel.model.levels.LogoSimulationLevelList
import fr.univ_artois.lgi2a.similar2logo.lib.model.ConeBasedPerceptionModel
import fr.univ_artois.lgi2a.similar2logo.lib.tools.html.Similar2LogoHtmlRunner
import fr.univ_artois.lgi2a.similar2logo.lib.tools.math.MeanAngle
import fr.univ_artois.lgi2a.similar2logo.kernel.tools.MathUtil

//Define the parameters of the simulation
def parameters = new LogoSimulationParameters() {
    
    @Parameter(name = "repulsion distance", description = "the repulsion distance")
    public double repulsionDistance = 1

    @Parameter(name = "orientation distance", description = "the orientation distance")
    public double orientationDistance = 2
    
    @Parameter(name = "attraction distance", description = "the attraction distance")
    public double attractionDistance = 4
    
    @Parameter(name = "repulsion weight", description = "the repulsion weight")
    public double repulsionWeight = 10

    @Parameter(name = "orientation weight", description = "the orientation weight")
    public double orientationWeight = 20
    
    @Parameter(name = "attraction weight", description = "the attraction weight")
    public double attractionWeight = 0.1

    @Parameter(name = "maximal initial speed", description = "the maximal initial speed")
    public double maxInitialSpeed = 2

    @Parameter(name = "minimal initial speed", description = "the minimal initial speed")
    public double minInitialSpeed = 1

    @Parameter(name = "perception angle", description = "the perception angle in rad")
    public double perceptionAngle = PI

    @Parameter(name = "number of agents", description = "the number of boids in the simulation")
    public int nbOfAgents = 2000

    @Parameter(name = "max angular speed", description = "the maximal angular speed in rad/step")
    public double maxAngle = PI/4
}

//Define the decision model of a boid
def decisionModel = new AbstractAgtDecisionModel(LogoSimulationLevelList.LOGO) {
    void decide(
        SimulationTimeStamp s, //the current simulation step
        SimulationTimeStamp ns, //the next simulation step
        IGlobalState gs, //the global state of the agent
        ILocalStateOfAgent pls, //the public local state of the boid
        ILocalStateOfAgent prls, //the private local state of the boid
        IPerceivedData pd, //the data perceived by the boid
        InfluencesMap i //the influences produced by the boid
    ) {
        if(!pd.turtles.empty) {
            def sc = 0, //the speed command
                meanAngle = new MeanAngle(),//the orientation command
                n = 0 //the number of boids in the orientation area
            //computes the commands according to the area in which the perceived boid is located 
            pd.turtles.each{ boid ->
                switch(boid.distanceTo) {
                    //the repulsion area
                    case {it <= parameters.repulsionDistance}:
                        meanAngle.add(pls.direction - boid.directionTo, parameters.repulsionWeight)
                        break
                    //the orientation area
                    case {it > parameters.repulsionDistance && it <= parameters.orientationDistance}:
                        meanAngle.add(boid.content.direction - pls.direction,parameters.orientationWeight)
                        sc+=boid.content.speed - pls.speed
                        n++
                        break
                    //the attraction area
                    case {it > parameters.orientationDistance && it <= parameters.attractionDistance}:
                        meanAngle.add(boid.directionTo- pls.direction, parameters.attractionWeight)
                        break
                }
            }
            //the orientation command
            def oc = meanAngle.value()
            if (!MathUtil.areEqual(oc, 0)) {
                //ceil the orientation command
                if(abs(oc) > parameters.maxAngle) oc = signum(oc)*parameters.maxAngle
                //emit a change direction influence
                i.add new ChangeDirection(s, ns, oc, pls)
            }
            //emit a change speed influence
            if (n > 0) i.add new ChangeSpeed(s, ns, sc/n, pls)
        }
    }
}

//Define the initial state of the simulation
def simulationModel = new AbstractLogoSimulationModel(parameters) {
    
    //Generate the agents
    protected AgentInitializationData generateAgents(
        ISimulationParameters p,
        Map<LevelIdentifier, ILevel> l
    ) {
        def result = new AgentInitializationData()
        //For each boid to be generated
        p.nbOfAgents.times {
            //generate the boid
            result.agents.add TurtleFactory.generate(
                new ConeBasedPerceptionModel(p.attractionDistance,p.perceptionAngle,true,false,false), //the perception model
                decisionModel, //the decision model
                new AgentCategory("b", TurtleAgentCategory.CATEGORY), //the category
                PRNG.get().randomAngle(), //the initial orientation
                p.minInitialSpeed + PRNG.get().randomDouble()*(p.maxInitialSpeed-p.minInitialSpeed),//the initial speed
                0,//the initial acceleration
                PRNG.get().randomDouble()*p.gridWidth,//the initial x position
                PRNG.get().randomDouble()*p.gridHeight //the initial y position
            )
        }
        return result
    }
}

// Creation of the runner
def runner = new Similar2LogoHtmlRunner( )

// Configuration of the runner
runner.config.exportAgents = true

// Initialize the runner
runner.initializeRunner simulationModel

// Open the GUI
runner.showView( )

Resolving dependency: fr.univ-artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT {default=[default]}
Preparing to download artifact fr.univ-artois.lgi2a#similar2logo-lib;1.0-SNAPSHOT!similar2logo-lib.jar
Preparing to download artifact fr.univ-artois.lgi2a#similar2logo-kernel;1.0-SNAPSHOT!similar2logo-kernel.jar
Preparing to download artifact com.sparkjava#spark-core;2.8.0!spark-core.jar(bundle)
Preparing to download artifact org.slf4j#slf4j-api;1.7.25!slf4j-api.jar
Preparing to download artifact org.slf4j#jul-to-slf4j;1.7.25!jul-to-slf4j.jar
Preparing to download artifact org.slf4j#jcl-over-slf4j;1.7.25!jcl-over-slf4j.jar
Preparing to download artifact org.slf4j#log4j-over-slf4j;1.7.25!log4j-over-slf4j.jar
Preparing to download artifact org.slf4j#slf4j-jdk14;1.7.25!slf4j-jdk14.jar
Preparing to download artifact org.apache.commons#commons-math3;3.6.1!commons-math3.jar
Preparing to download artifact fr.univ-artois.lgi2a#similar-microKernel;1.0-SNAPSHOT!similar-microKernel.jar
Preparing to downlo

No Outputs